# সেম্যান্টিক কের্নেলে এজেন্ট স্ক্র্যাচপ্যাড সহ চ্যাট ইতিহাস সংক্ষেপণ

এই নোটবুকটি দেখায় কীভাবে সেম্যান্টিক কের্নেলের চ্যাট ইতিহাস সংক্ষেপণ বৈশিষ্ট্য ব্যবহার করা যায় এবং এজেন্ট স্ক্র্যাচপ্যাডের মাধ্যমে কথোপকথনের প্রেক্ষাপট বজায় রাখা যায়। এটি দক্ষ AI এজেন্ট তৈরি করার জন্য গুরুত্বপূর্ণ, যা দীর্ঘ কথোপকথন পরিচালনা করতে পারে টোকেন সীমা অতিক্রম না করে।

## আপনি কী শিখবেন:
1. **চ্যাট ইতিহাস সংক্ষেপণ**: টোকেন ব্যবহারের নিয়ন্ত্রণে কথোপকথনের ইতিহাস স্বয়ংক্রিয়ভাবে সংক্ষেপ করার পদ্ধতি
2. **এজেন্ট স্ক্র্যাচপ্যাড**: ব্যবহারকারীর পছন্দ এবং সম্পন্ন কাজগুলি ট্র্যাক করার জন্য একটি স্থায়ী মেমরি সিস্টেম
3. **টোকেন ব্যবহারের ট্র্যাকিং**: ইতিহাস সংক্ষেপণের সাথে এবং ছাড়া টোকেন ব্যবহারের পরিবর্তন পর্যবেক্ষণ

## পূর্বশর্ত:
- পরিবেশ ভেরিয়েবল কনফিগার করা সহ Azure OpenAI সেটআপ
- পূর্ববর্তী পাঠ থেকে এজেন্টের মৌলিক ধারণাগুলির বোঝাপড়া


## প্রয়োজনীয় প্যাকেজগুলি আমদানি করুন


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## এজেন্ট স্ক্র্যাচপ্যাড বোঝা

### এজেন্ট স্ক্র্যাচপ্যাড কী?

একটি **এজেন্ট স্ক্র্যাচপ্যাড** হলো একটি স্থায়ী মেমরি সিস্টেম যা এজেন্ট ব্যবহার করে:
- **সম্পন্ন কাজের ট্র্যাক রাখা**: ব্যবহারকারীর জন্য কী কী কাজ সম্পন্ন হয়েছে তা রেকর্ড করা
- **ব্যবহারকারীর পছন্দ সংরক্ষণ**: পছন্দ, অপছন্দ এবং প্রয়োজনীয়তাগুলো মনে রাখা
- **প্রসঙ্গ বজায় রাখা**: কথোপকথনের মধ্যে গুরুত্বপূর্ণ তথ্য সহজলভ্য রাখা
- **অতিরিক্ত পুনরাবৃত্তি কমানো**: একই প্রশ্ন বারবার না করা

### এটি কীভাবে কাজ করে:
1. **লেখার অপারেশন**: এজেন্ট নতুন তথ্য শিখলে স্ক্র্যাচপ্যাড আপডেট করে
2. **পড়ার অপারেশন**: সিদ্ধান্ত নেওয়ার সময় এজেন্ট স্ক্র্যাচপ্যাড ব্যবহার করে
3. **স্থায়িত্ব**: তথ্য সংরক্ষিত থাকে, এমনকি চ্যাট ইতিহাস কমে গেলেও

এটি এজেন্টের ব্যক্তিগত নোটবুকের মতো, যা কথোপকথনের ইতিহাসকে সম্পূরক করে।


## পরিবেশ কনফিগারেশন


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## এজেন্ট স্ক্র্যাচপ্যাড প্লাগইন তৈরি করুন

এই প্লাগইনটি এজেন্টকে একটি স্থায়ী স্ক্র্যাচপ্যাড ফাইলে পড়া এবং লেখা করার সুযোগ দেয়।


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## চ্যাট ইতিহাস রিডিউসার আরম্ভ করুন

ChatHistorySummarizationReducer স্বয়ংক্রিয়ভাবে কথোপকথনের ইতিহাস সংক্ষিপ্ত করে যখন এটি একটি নির্দিষ্ট সীমা অতিক্রম করে।


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## ভ্যাকেশন পরিকল্পনা এজেন্ট তৈরি করুন

এই এজেন্ট ব্যবহারকারীদের ভ্যাকেশন পরিকল্পনা করতে সাহায্য করবে এবং স্ক্র্যাচপ্যাডের মাধ্যমে প্রসঙ্গ বজায় রাখবে।


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## ছুটির পরিকল্পনার কথোপকথন চালান

এখন একটি সম্পূর্ণ কথোপকথন চালিয়ে দেখুন যা প্রদর্শন করে:
1. প্রাথমিক পরিকল্পনার অনুরোধ
2. পছন্দ সংগ্রহ
3. ভ্রমণসূচি তৈরি
4. অবস্থান পরিবর্তন
5. চ্যাট ইতিহাস সংক্ষিপ্তকরণ
6. স্ক্র্যাচপ্যাড ব্যবহার


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## ফলাফল বিশ্লেষণ

চলুন দেখি আমাদের কথোপকথনের সময় কী ঘটেছিল:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## মূল বিষয়বস্তু

### ১. চ্যাট ইতিহাস কমানো
- **স্বয়ংক্রিয় কার্যক্রম**: বার্তার সংখ্যা নির্ধারিত সীমা অতিক্রম করলে ইতিহাস কমানো শুরু হয়
- **টোকেন সঞ্চয়**: সারসংক্ষেপের পর টোকেন ব্যবহারে উল্লেখযোগ্য হ্রাস ঘটে
- **প্রাসঙ্গিকতা বজায় রাখা**: গুরুত্বপূর্ণ তথ্য সারসংক্ষেপে সংরক্ষিত থাকে

### ২. এজেন্ট স্ক্র্যাচপ্যাডের সুবিধা
- **স্থায়ী স্মৃতি**: ব্যবহারকারীর পছন্দ ইতিহাস কমানোর পরেও সংরক্ষিত থাকে
- **কাজের ট্র্যাকিং**: এজেন্ট সম্পন্ন কাজের রেকর্ড রাখে
- **উন্নত অভিজ্ঞতা**: পছন্দ বারবার জানাতে হয় না

### ৩. টোকেন ব্যবহারের ধরণ
- **রৈখিক বৃদ্ধি**: প্রতিটি বার্তার সাথে টোকেন সংখ্যা বাড়ে
- **নাটকীয় হ্রাস**: ইতিহাস কমানোর ফলে টোকেন সংখ্যা উল্লেখযোগ্যভাবে কমে যায়
- **টেকসই কথোপকথন**: সীমার মধ্যে দীর্ঘ সময় ধরে কথোপকথন চালিয়ে যাওয়া সম্ভব


## পরিস্কার

এই ডেমোর সময় তৈরি করা স্ক্র্যাচপ্যাড ফাইলটি মুছে ফেলুন:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# সারাংশ

অভিনন্দন! আপনি সফলভাবে একটি AI এজেন্ট তৈরি করেছেন যা উন্নত প্রসঙ্গ ব্যবস্থাপনার ক্ষমতা রাখে:

## আপনি যা শিখেছেন:
- **চ্যাট ইতিহাস সংক্ষেপণ**: কথোপকথন স্বয়ংক্রিয়ভাবে সংক্ষেপণ করে টোকেন সীমা পরিচালনা করা
- **এজেন্ট স্ক্র্যাচপ্যাড**: ব্যবহারকারীর পছন্দ এবং সম্পন্ন কাজের জন্য স্থায়ী মেমোরি প্রয়োগ করা
- **টোকেন ব্যবস্থাপনা**: দীর্ঘ কথোপকথনে টোকেন ব্যবহারের ট্র্যাক রাখা এবং অপ্টিমাইজ করা
- **প্রসঙ্গ সংরক্ষণ**: কথোপকথন সংক্ষেপণের সময় গুরুত্বপূর্ণ তথ্য ধরে রাখা

## বাস্তব জীবনের প্রয়োগ:
- **গ্রাহক সেবা বট**: সেশনের মধ্যে গ্রাহকের পছন্দ মনে রাখা
- **ব্যক্তিগত সহকারী**: চলমান প্রকল্প এবং ব্যবহারকারীর অভ্যাস ট্র্যাক করা
- **শিক্ষামূলক টিউটর**: শিক্ষার্থীর অগ্রগতি এবং শেখার পছন্দ সংরক্ষণ করা
- **স্বাস্থ্যসেবা সহকারী**: রোগীর ইতিহাস সংরক্ষণ করা, টোকেন সীমা মেনে চলা

## পরবর্তী পদক্ষেপ:
- আরও উন্নত স্ক্র্যাচপ্যাড স্কিমা প্রয়োগ করা
- বহু-ব্যবহারকারী পরিস্থিতির জন্য ডাটাবেস সংরক্ষণ যোগ করা
- ডোমেইন-নির্দিষ্ট প্রয়োজনের জন্য কাস্টম সংক্ষেপণ কৌশল তৈরি করা
- সেমান্টিক মেমোরি অনুসন্ধানের জন্য ভেক্টর ডাটাবেসের সাথে সংযুক্ত করা
- এমন এজেন্ট তৈরি করা যা কয়েকদিন পরে পূর্ণ প্রসঙ্গ সহ কথোপকথন পুনরায় শুরু করতে পারে



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
